In [1]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoStore

In [2]:
loader = TweetLoader()
loader.extract_tweets(count=5)

In [3]:
loader.get_loaded_tweets_as_json()

[{'created_at': 'Mon Mar 06 17:26:38 +0000 2023',
  'id': 1632794833747099648,
  'id_str': '1632794833747099648',
  'text': 'German Chancellor Olaf Scholz risks derailing the EU’s green deal with his government’s push to allow combustion ca… https://t.co/gM5a537TG7',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/gM5a537TG7',
     'expanded_url': 'https://twitter.com/i/web/status/1632794833747099648',
     'display_url': 'twitter.com/i/web/status/1…',
     'indices': [117, 140]}]},
  'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 34713362,
   'id_str': '34713362',
   'name': 'Bloomberg',
   'screen_name': 'business',
   'location': 'New York and the World',
   'description': 'The first word

In [7]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Bloomberg
Fox News
Fox News
Forbes
Bloomberg


In [8]:
store = MongoStore("twitter_dashboard_db", "home_timeline")

In [9]:
store.save_tweets(loader.tweets)

1. business
2. FoxNews
3. FoxNews
4. Forbes
5. business


In [10]:
stored_tweets = store.load_mongo_data()
len(stored_tweets)

3